In [3]:
from IPython.display   import Image , display
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph  import StateGraph   , START , END
from langgraph.graph import MessagesState
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode

import os 
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
model = ChatGroq(
    model="llama-3.1-8b-instant",   
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

In [5]:
@tool
def multiply(x:int , y:int)-> int:
    """Returns the multiplication of given two integers"""
    return x*y

@tool
def add(x:int , y:int)->int:
    """Returns the addition of given two integers """
    return x+y 

@tool
def divide(x:int , y:int)-> int:
    """Returns the division of given two integers"""
    return x/y

In [6]:
tools=[multiply , add , divide]
llm_with_tools= model.bind_tools(tools)

In [7]:
sys_msg=SystemMessage(content= "You are a helpful assisstant tasked with performing arithmetic actions on a  given set of inputs")

In [8]:
def assistant(state:MessagesState):
    return {"messages":[llm_with_tools.invoke([sys_msg] + state["messages"] ) ] }


In [9]:
builder= StateGraph(MessagesState)
builder.add_node("tools" , ToolNode(tools))
builder.add_node("assistant" , assistant)

builder.add_edge(START , "assistant")
builder.add_conditional_edges("assistant" , tools_condition)
builder.add_edge("tools" , "assistant")
react_graph=builder.compile()

display(Image(react_graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [10]:
messages = [HumanMessage(content= "Add 4 and 2 , divide the output by 2 and then multiply it by 5 ")]
result= react_graph.invoke({"messages": messages})

In [13]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 4 and 2 , divide the output by 2 and then multiply it by 5 
================================== Ai Message ==================================
Tool Calls:
  add (x53m6n1fs)
 Call ID: x53m6n1fs
  Args:
    x: 4
    y: 2
  divide (zckrr5e01)
 Call ID: zckrr5e01
  Args:
    x: 6
    y: 2
  multiply (q7tmssg2d)
 Call ID: q7tmssg2d
  Args:
    x: 3
    y: 5
================================= Tool Message =================================
Name: add

6
================================= Tool Message =================================
Name: divide

3.0
================================= Tool Message =================================
Name: multiply

15
================================== Ai Message ==================================

The final result is 15.
